<h1>Test Tweet</h1>

<h2>Input Text</h2>

In [1]:
text="Bpk @ridwankamil43 tidak sebaik dahulu 20 ridwan43@gmail.com pic.twitter.com/DDAFssa232dsa #ridwangembel"

<h2>Preprocessing</h2>

In [2]:
import re
kumpText=[]
def preprocessing(text="",
                  casefolding=True,
                  removeusername=True,
                  removenonalphabet=True,
                  removestopword=True,
                  stemming=True,
                  removehashtag=True,
                  removeemail=True,
                  removeAdr=True):
    
    if(text!=""):
        import re
       
        if(casefolding):
            text = text.lower()
            print("Case Folding = "+text)
            
        if(removeusername):
            nama = re.compile("(?:^|\s)[＠ @]{1}([^\s#<>[\]|{}]+)", re.UNICODE)
            text = re.sub(nama, ' ', text)
            print("Remove Username = "+text)

        
        if(removeAdr):
            com= re.compile(r' \w+\.\w+\.*\w*\D\w*')
            text = re.sub(com, ' ', text)
            print("Remove Address = "+text)

            
        if(removehashtag):
            hashtag = re.compile("(?:^|\s)[＃#]{1}(\w+)", re.UNICODE)
            text = re.sub(hashtag, ' ', text)
            print("Remove Hashtag = "+text)
 
        if(removeemail):
            email = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
            text = re.sub(email, ' ', text) 
            print("Remove Email = "+text)
            
        if(removenonalphabet):
            import re
            text = ' '.join(re.findall(r'\b[a-z]+?[a-z]+\b',text))
            print("Remove non Alphabet = "+text)
    
        if(removestopword):
            from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
            factory = StopWordRemoverFactory()
            Sastrawi_StopWords_id = factory.get_stop_words()
            temp = [t for t in re.findall(r'\b[a-z]+-?[a-z]+\b',text) if t not in Sastrawi_StopWords_id]
            text = ' '.join(temp)
            print("Remove Stopword = "+text)
            
        if(stemming):
            from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
            stemmer = StemmerFactory().create_stemmer()
            text   = stemmer.stem(text)
            print("Stemming = "+text)
            
    return(text)


<h2>Mengolah Teks dan Normalisasi Kata</h2>

In [3]:
bersih=preprocessing(text)

file = open("kamus.txt", encoding="utf-8", errors='replace')
Teks = file.readlines()
Teks_string=' '.join(Teks)
fix=eval(Teks_string)


t=" ",bersih," "
t_string=' '.join(t)
t_jadi=t_string 
    
for slang, formal in fix.items():
    t_jadi = t_jadi.replace(slang,formal)
print(t_jadi)

Case Folding = bpk @ridwankamil43 tidak sebaik dahulu 20 ridwan43@gmail.com pic.twitter.com/ddafssa232dsa #ridwangembel
Remove Username = bpk  tidak sebaik dahulu 20 ridwan43@gmail.com pic.twitter.com/ddafssa232dsa #ridwangembel
Remove Address = bpk  tidak sebaik dahulu 20 ridwan43@gmail.com  #ridwangembel
Remove Hashtag = bpk  tidak sebaik dahulu 20 ridwan43@gmail.com  
Remove Email = bpk  tidak sebaik dahulu 20    
Remove non Alphabet = bpk tidak sebaik dahulu
Remove Stopword = bpk tidak sebaik
Stemming = bpk tidak baik
  bapak tidak baik  


<h2>Negasi</h2>

In [4]:
import nlp_if_lib as nlp
import urllib, json
import requests
from spacy.lang.id import Indonesian
nlp_id_spacy = Indonesian() 

teksAnt=[]

kataStrip=t_jadi.strip()
kataToken = nlp_id_spacy(kataStrip)  
token_pos = nlp.getPOSTag()
    
for x in kataToken:
    teksAnt.append(x.text)
    
lenTeksAnt=len(teksAnt)   
for y in range(0,lenTeksAnt):
    antonim=[]
    if(teksAnt[y]=="tidak" or teksAnt[y]=="kurang" or teksAnt[y]=="belum"):
        for token in kataToken:
            t=token.lemma_
            try: 
                pos=token_pos[token.lemma_]
                if(token_pos[teksAnt[y+1]]=="adj"):
                    url = "http://kateglo.com/api.php?format=json&phrase="+teksAnt[y+1]
                    response = requests.get(url)
                    data=response.text
                    parsed=json.loads(data)
                    post = parsed["kateglo"]["relation"]["a"]
                    lenPost=len(post)
                    for x in range(0,lenPost):
                        st=str(x)
                        value=post[st]
                        if(value['rel_type']=='a'):
                            if(value['lex_class']=='adj'):
                                ant=value['related_phrase']
                                antonim.append(ant)
                                teksAnt[y+1]=""
                                teksAnt[y]=""

            except:
                pos='n'
            
                
            
    listAnt=len(antonim)
    for i in range(0,listAnt):
        teksAnt.append(antonim[i])
        
ubahAnt=' '.join(teksAnt)
kataJadi=ubahAnt
print(kataJadi)

bapak   buruk jahat zalim


<h2>Load Model</h2>

In [5]:
import pickle


filename='MNB.model'

file=open('model\\'+filename,'rb')
bobot_model=pickle.load(file)
model=pickle.load(file)

print(bobot_model)
print(model)

print('oke')

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
oke


<h2>Pembobotan</h2>

In [6]:
bobot=bobot_model.transform([kataJadi])
matrix_bobot=bobot.toarray()
x_test=matrix_bobot

doc = 0
feature_names=bobot_model.get_feature_names()
feature_index=bobot[doc,:].nonzero()[1]
tfidf_scores=zip(feature_index,[bobot[doc,x] for x in feature_index])

for w, s in [(feature_names[i],s) for (i,s) in tfidf_scores]:
    print(w,s)

bapak 1.0


<h2>Prediksi</h2>

In [7]:
predict=model.predict(x_test)
prob_n, prob_p=model.predict_proba(x_test)[0]

print("positif :"+str(prob_p))
print("negatif :"+str(prob_n))

positif :0.5200499376790843
negatif :0.47995006232091586


<h2>Hasil</h2>

In [8]:
print(text)
print(predict[0])
if(predict[0]==1):
    print("positif")
else:
    print("negatif")

Bpk @ridwankamil43 tidak sebaik dahulu 20 ridwan43@gmail.com pic.twitter.com/DDAFssa232dsa #ridwangembel
1
positif
